<a href="https://colab.research.google.com/github/Shafaqulhamim/MS-coco-training/blob/main/MS_COCO_Dataset_Download_and_traininng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless==4.5.4.60

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47.6 MB 2.0 MB/s 


In [ ]:
!pip install fiftyone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 15.8 MB/s 
     |████████████████████████████████| 226 kB 50.3 MB/s 
     |████████████████████████████████| 47 kB 4.8 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 61 kB 322 kB/s 
     |████████████████████████████████| 29.2 MB 65.4 MB/s 
     |████████████████████████████████| 133 kB 59.5 MB/s 
     |████████████████████████████████| 563 kB 53.9 MB/s 
     |████████████████████████████████| 508 kB 10.1 MB/s 
     |████████████████████████████████| 135 kB 46.1 MB/s 
     |████████████████████████████████| 13.2 MB 46.0 MB/s 
     |████████████████████████████████| 106 kB 70.2 MB/s 
     |████████████████████████████████| 79.9 MB 123 kB/s 
     |████████████████████████████████| 132 kB 43.4 MB/s 
     |███████████████████████████████

In [ ]:
!pip install torch torchvision

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

dataset_train = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    label_types=["detections"],
    classes=['person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'boat',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'bottle',
 'chair',
 'couch',
 'potted plant',
 'dining table',
 'tv'],
     max_samples=20000,

)
# session = fo.launch_app(dataset_train)



Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'
15194 images found; downloading the remaining 4806
 100% |████████████████| 4806/4806 [3.3m elapsed, 0s remaining, 23.2 images/s]      
Writing annotations for 20000 downloaded samples to '/root/fiftyone/coco-2017/train/labels.json'
Dataset info written to '/root/fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'train'
 100% |█████████████| 20000/20000 [2.7m elapsed, 0s remaining, 142.2 samples/s]      
Dataset 'coco-2017-train-20000' created


In [ ]:
dataset_train

Name:        coco-2017-train-20000
Media type:  image
Num samples: 20000
Persistent:  False
Tags:        ['train']
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

In [ ]:
import fiftyone.utils.coco as fouc

In [ ]:
import torch

In [ ]:
class FiftyOneTorchDataset(torch.utils.data.Dataset):
    """A class to construct a PyTorch dataset from a FiftyOne dataset.
    
    Args:
        fiftyone_dataset: a FiftyOne dataset or view that will be used for training or testing
        transforms (None): a list of PyTorch transforms to apply to images and targets when loading
        gt_field ("ground_truth"): the name of the field in fiftyone_dataset that contains the 
            desired labels to load
        classes (None): a list of class strings that are used to define the mapping between
            class names and indices. If None, it will use all classes present in the given fiftyone_dataset.
    """

    def __init__(
        self,
        fiftyone_dataset,
        transforms=None,
        gt_field="ground_truth",
        classes=None,
    ):
        self.samples = fiftyone_dataset
        self.transforms = transforms
        self.gt_field = gt_field

        self.img_paths = self.samples.values("filepath")

        self.classes = classes
        if not self.classes:
            # Get list of distinct labels that exist in the view
            self.classes = self.samples.distinct(
                "%s.detections.label" % gt_field
            )

        if self.classes[0] != "background":
            self.classes = ["background"] + self.classes

        self.labels_map_rev = {c: i for i, c in enumerate(self.classes)}

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        sample = self.samples[img_path]
        metadata = sample.metadata
        img = Image.open(img_path).convert("RGB")

        boxes = []
        labels = []
        area = []
        iscrowd = []
        detections = sample[self.gt_field].detections
        for det in detections:
            category_id = self.labels_map_rev[det.label]
            coco_obj = fouc.COCOObject.from_label(
                det, metadata, category_id=category_id,
            )
            x, y, w, h = coco_obj.bbox
            boxes.append([x, y, x + w, y + h])
            labels.append(coco_obj.category_id)
            area.append(coco_obj.area)
            iscrowd.append(coco_obj.iscrowd)

        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        target["image_id"] = torch.as_tensor([idx])
        target["area"] = torch.as_tensor(area, dtype=torch.float32)
        target["iscrowd"] = torch.as_tensor(iscrowd, dtype=torch.int64)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.img_paths)

    def get_classes(self):
        return self.classes

In [ ]:
sample = dataset_train.first()
print(sample.ground_truth.detections[0])

<Detection: {
    'id': '62a8f0b8b473aace1e335a6b',
    'attributes': BaseDict({}),
    'tags': BaseList([]),
    'label': 'potted plant',
    'bounding_box': BaseList([
        0.32009375,
        0.07247663551401869,
        0.39825,
        0.7572897196261682,
    ]),
    'mask': None,
    'confidence': None,
    'index': None,
    'supercategory': 'furniture',
    'iscrowd': 0,
}>


In [ ]:
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_data_loader = DataLoader(
    dataset_train,
    batch_size=10,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
import torch

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re

from PIL import Image
from matplotlib import pyplot as plt


In [ ]:
dataset_train

In [ ]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

In [ ]:
import torchvision

In [ ]:
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)

In [ ]:
# num_classes = 21
# in_features = model.roi_heads.box_predictor.cls_score.in_features
# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
# model.to(device)
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# # lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
# lr_scheduler = None

# num_epochs = 2

In [ ]:
def get_model(num_classes):
    # load a model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
    
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [ ]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../


Cloning into 'vision'...
remote: Enumerating objects: 156554, done.
remote: Counting objects: 100% (1118/1118), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 156554 (delta 986), reused 1092 (delta 979), pack-reused 155436
Receiving objects: 100% (156554/156554), 311.35 MiB | 25.66 MiB/s, done.
Resolving deltas: 100% (138994/138994), done.
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be376084d version check against PyTorch's CUDA version


In [ ]:
# Import functions from the torchvision references we cloned
from engine import train_one_epoch, evaluate
import utils

def do_training(model, torch_dataset, torch_dataset_test, num_epochs=4):
    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        torch_dataset, batch_size=2, shuffle=True, num_workers=2,
        collate_fn=utils.collate_fn)
    
    data_loader_test = torch.utils.data.DataLoader(
        torch_dataset_test, batch_size=1, shuffle=False, num_workers=2,
        collate_fn=utils.collate_fn)

    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print("Using device %s" % device)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                    step_size=3,
                                                    gamma=0.1)

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)

        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

In [ ]:
from fiftyone import ViewField as F

busy_view = dataset_train.match(F("ground_truth.detections").length() > 10)

busy_torch_dataset = FiftyOneTorchDataset(busy_view)

In [ ]:
from fiftyone import ViewField as F

My_Classes = ['person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'boat',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'bottle',
 'chair',
 'couch',
 'potted plant',
 'dining table',
 'tv']
class_view = dataset_train.filter_labels("ground_truth",
        F("label").is_in(My_Classes))

print(len(class_view))

20000


In [ ]:
import transforms as T

train_transforms = T.Compose([T.ToTensor(), T.RandomHorizontalFlip(0.5)])
test_transforms = T.Compose([T.ToTensor()])

In [ ]:
train_view = class_view.take(16000, seed=51)
test_view = class_view.exclude([s.id for s in train_view])


In [ ]:
torch_dataset = FiftyOneTorchDataset(train_view, train_transforms,
        classes=My_Classes)
torch_dataset_test = FiftyOneTorchDataset(test_view, test_transforms, 
        classes=My_Classes)

In [ ]:
model = get_model(len(My_Classes)+1)

In [ ]:
do_training(model, torch_dataset, torch_dataset_test, num_epochs=2)

Using device cpu


/usr/local/lib/python3.7/dist-packages/pymongo/topology.py:165: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: https://pymongo.readthedocs.io/en/stable/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.7/dist-packages/pymongo/topology.py:165: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: https://pymongo.readthedocs.io/en/stable/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


Epoch: [0]  [   0/8000]  eta: 4 days, 17:33:36  lr: 0.000010  loss: 3.8570 (3.8570)  loss_classifier: 3.0781 (3.0781)  loss_box_reg: 0.0592 (0.0592)  loss_objectness: 0.6891 (0.6891)  loss_rpn_box_reg: 0.0305 (0.0305)  time: 51.1021  data: 1.8485  max mem: 0
Epoch: [0]  [  10/8000]  eta: 4 days, 11:28:01  lr: 0.000060  loss: 3.8294 (3.8663)  loss_classifier: 3.0694 (3.0656)  loss_box_reg: 0.0226 (0.0422)  loss_objectness: 0.6907 (0.6913)  loss_rpn_box_reg: 0.0328 (0.0673)  time: 48.4207  data: 0.1846  max mem: 0
Epoch: [0]  [  20/8000]  eta: 4 days, 7:18:33  lr: 0.000110  loss: 3.8372 (3.8880)  loss_classifier: 3.0227 (3.0043)  loss_box_reg: 0.0337 (0.0471)  loss_objectness: 0.6899 (0.6901)  loss_rpn_box_reg: 0.0678 (0.1464)  time: 46.3809  data: 0.0136  max mem: 0
Epoch: [0]  [  30/8000]  eta: 4 days, 3:45:25  lr: 0.000160  loss: 3.5702 (3.7310)  loss_classifier: 2.7962 (2.8743)  loss_box_reg: 0.0337 (0.0413)  loss_objectness: 0.6886 (0.6893)  loss_rpn_box_reg: 0.0866 (0.1261)  time: 

KeyboardInterrupt: ignored

In [ ]:
# class Averager:
#     def __init__(self):
#         self.current_total = 0.0
#         self.iterations = 0.0

#     def send(self, value):
#         self.current_total += value
#         self.iterations += 1

@property
#     def value(self):
#         if self.iterations == 0:
#             return 0
#         else:
#             return 1.0 * self.current_total / self.iterations

#     def reset(self):
#         self.current_total = 0.0
#         self.iterations = 0.0

In [ ]:
# loss_hist = Averager()
# itr = 1

# for epoch in range(num_epochs):
#     loss_hist.reset()
    
#     for images, targets, image_ids in train_data_loader:
        
#         images = list(image.to(device) for image in images)
#         targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

#         loss_dict = model(images, targets)

#         losses = sum(loss for loss in loss_dict.values())
#         loss_value = losses.item()

#         loss_hist.send(loss_value)

#         optimizer.zero_grad()
#         losses.backward()
#         optimizer.step()

#         if itr % 50 == 0:
#             print(f"Iteration #{itr} loss: {loss_value}")

#         itr += 1
    
#     # update the learning rate
#     if lr_scheduler is not None:
#         lr_scheduler.step()

#     print(f"Epoch #{epoch} loss: {loss_hist.value}")  

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


ValueError: ignored